<a href="https://github.com/PaddlePaddle/PaddleSpeech"><img style="position: absolute; z-index: 999; top: 0; right: 0; border: 0; width: 128px; height: 128px;" src="https://nosir.github.io/cleave.js/images/right-graphite@2x.png" alt="Fork me on GitHub"></a>


# 语音识别——DeepSpeech2
  
# 0. 视频理解与字幕

In [1]:
# 下载demo视频
!test -f work/source/subtitle_demo1.mp4 || wget https://paddlespeech.bj.bcebos.com/demos/asr_demos/subtitle_demo1.mp4 -P work/source/

In [2]:
import IPython.display as dp
from IPython.display import HTML
html_str = '''
<video controls width="600" height="360" src="{}">animation</video>
'''.format("work/source/subtitle_demo1.mp4 ")
dp.display(HTML(html_str))
print ("ASR结果为：当我说我可以把三十年的经验变成一个准确的算法他们说不可能当我说我们十个人就能实现对十九个城市变电站七乘二十四小时的实时监管他们说不可能")

> Demo实现：[Dhttps://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/automatic_video_subtitiles/](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/demos/automatic_video_subtitiles/)


# 1. 前言

## 1.1 背景知识
语音识别(Automatic Speech Recognition, ASR) 是一项从一段音频中提取出语言文字内容的任务。

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/0231a71b0617485d85586d232f65db6379115befdf014068bd90fb15c5786c94"/>
<br>
(出处：DLHLP 李宏毅 语音识别课程PPT)
</div>

目前该技术已经广泛应用于我们的工作和生活当中，包括生活中使用手机的语音转写，工作上使用的会议记录等等。

## 1.2 发展历史


* 早期，生成模型流行阶段：GMM-HMM (上世纪90年代)
* 深度学习爆发初期： DNN，CTC[1] （2006）
* RNN流行，Attention提出初期: RNN-T[2]（2013）, DeepSpeech[3] (2014)， DeepSpeech2 [4] (2016)， LAS[5]（2016）
* Attetion is all you need提出开始[6]: Transformer[6]（2017），Transformer-transducer[7]（2020） Conformer[8] （2020

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/d6060426bba341a187422803c0f8ac2e2162c5c5422e4070a3425c09f7801379" height=1300, width=1000 />
</div>

Deepspeech2模型包含了CNN，RNN，CTC等深度学习语音识别的基本技术，因此本教程采用了Deepspeech2作为讲解深度学习语音识别的开篇内容。

# 2. 实战：使用 DeepSpeech2 进行语音识别的流程

Deepspeech2 模型，其主要分为3个部分：
1. 特征提取模块：此处使用 linear 特征，也就是将音频信息由时域转到频域后的信息。
2. Encoder：多层神经网络，用于对特征进行编码。
3. CTC Decoder： 采用了 CTC 损失函数训练；使用 CTC 解码得到结果。

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/f7268c47d55e487cbb97f123785cab248f4371072381465a8b43af33f4abdf83" height=1000, width=800/>
</div>

## 2.1 Deepspeech2 模型结构

### 2.1.1 Encoder


Encoder 主要采用了 2 层降采样的 CNN（subsampling Convolution layer）和多层 RNN（Recurrent Neural Network）层组成。

其中降采样的 CNN 主要用途在提取局部特征，减少模型输入的帧数，降低计算量，并易于模型收敛。


  
#### 2.1.1.1 CNN: Receptive field

假如以 $F_j$ 代表 $L_j$ 的 cnn 滤波器大小, $S_i$ 代表 $L_i$ 的CNN滤波器跳跃长度，并设定 $S_0 = 1$。那么 $L_k$ 的感受野大小可以由以下公式计算：

$$\boxed{R_k = 1 + \sum_{j=1}^{k} [(F_j - 1) \prod_{i=0}^{j-1} S_i]}$$
在下面的例子中, $F_1 = F_2 = 3$ 并且 $S_1 = S_2 = 2$, 因此可以得到 $R_2 = 1 + 2\cdot 1 + 2\cdot 2 = 7$

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/b3c53b7d1b2944acb91520454f5b27be0e0e0af110e24959942d85e87992d6d0"  />
</div>

  
#### 2.1.1.2 RNN

  而多层 RNN 的作用在于获取语音的上下文信息，这样可以获得更加准确的信息，并一定程度上进行语义消歧。
  
Deepspeech2 的模型中 RNNCell 可以选用 GRU 或者 LSTM。
  

#### 2.1.1.3 Softmax
而最后 softmax 层将特征向量映射到为一个字表长度的向量，向量中存储了当前 step 结果预测为字表中每个字的概率。


### 2.1.2 Decoder
Decoder 的作用主要是将 Encoder 输出的概率解码为最终的文字结果。

对于 CTC 的解码主要有3种方式：

* CTC greedy search 

* CTC beam search 

* CTC Prefix beam search

#### 2.1.2.1 CTC Greedy Search

在每个时间点选择后验概率最大的 label 加入候选序列中，最后对候选序列进行后处理，就得到解码结果。

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/d6e90cf5d20047ddbcdba5ff13c6c96a9ffca20faaa84927a6012fa1927e5b8d" height=800, width=500 />
</div>


#### 2.1.2.2 CTC Beam Search

CTC Beam Search 的方式是有 beam size 个候选序列，并在每个时间点生成新的最好的 beam size 个候选序列。
最后在 beam size 个候选序列中选择概率最高的序列生成最终结果。


<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/98560261bc584c209a339396296d56559c6daa9a9bb74532b014c6d7ee4b7a5f" height=1200, width=800 />
  <br>
  引用自[9]
</div>

#### 2.1.2.3 CTC Prefix Beam Search

CTC prefix beam search和 CTC beam search 的主要区别在于：

CTC beam search 在解码过程中产生的候选有可能产生重复项，而这些重复项在 CTC beam search 的计算过程中是各自独立的，占用了 beam 数，降低解码的多样性和鲁棒性。

而 CTC prefix beam search 在解码过程中合并了重复项的概率，提升解码的鲁棒性和多样性。

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/3d8c949abd0948f1be35ad2b3cf9a0100be1dcc771724d9681b5b31fc1fd4fa6" height=1200, width=800  />
  <br>
  引用自[9]
</div>

CTC prefix beam search 计算过程如下图所示：

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/7a779a8687054d559515153d9ab35aa5e9f54676f5b64984960c62f6998a0ce3" height=1200, width=800  />
  <br>
  引用自[10]
</div>


> [CTCLoss](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/topic/ctc/) 相关介绍参看 [Topic](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/topic/) 内容。

#### 2.1.2.4 使用 N-gram 语言模型

对于解码的候选结果的打分，除了有声学模型的分数外，还会有额外的语言模型分以及长度惩罚分。


设定 $W$ 为解码结果，$X$ 为输入语音， $\alpha$ 和 $\beta$ 为设定的超参数。
则最终分数的计算公式为:
$$
score = P_{am}(W \mid X) \cdot P_{lm}(W) ^ \alpha \cdot |W|^\beta
$$



## 2.2 准备工作
    


### 2.2.1 安装 paddlespeech
    


In [3]:
!pip install --upgrade pip && pip install paddlespeech==0.1.0


  
### 2.2.2 准备工作目录



In [4]:
!mkdir -p ./work/workspace_asr_ds2
%cd ./work/workspace_asr_ds2


### 2.2.3 获取预训练模型和相关文件
  


In [5]:
!test -f ds2.model.tar.gz || wget -nc https://paddlespeech.bj.bcebos.com/s2t/aishell/asr0/ds2.model.tar.gz
!tar xzvf ds2.model.tar.gz

In [6]:
# 构建一个数据增强的配置文件，由于预测不需要数据增强，因此文件为空即可
!touch conf/augmentation.json
# 下载语言模型
!mkdir -p data/lm
!test -f ./data/lm/zh_giga.no_cna_cmn.prune01244.klm || wget -nc https://deepspeech.bj.bcebos.com/zh_lm/zh_giga.no_cna_cmn.prune01244.klm -P data/lm
# 获取用于预测的音频文件
!test -f ./data/demo_01_03.wav || wget -nc https://paddlespeech.bj.bcebos.com/datasets/single_wav/zh/demo_01_03.wav -P ./data/

In [7]:
import IPython
IPython.display.Audio('./data/demo_01_03.wav')

In [8]:
# 快速体验识别结果
!paddlespeech asr --input ./data/demo_01_03.wav



### 2.2.4 导入python包
    


In [9]:
import paddle
import warnings
warnings.filterwarnings('ignore')

from yacs.config import CfgNode

from paddlespeech.s2t.frontend.speech import SpeechSegment
from paddlespeech.s2t.frontend.normalizer import FeatureNormalizer
from paddlespeech.s2t.frontend.featurizer.audio_featurizer import AudioFeaturizer
from paddlespeech.s2t.frontend.featurizer.text_featurizer import TextFeaturizer

from paddlespeech.s2t.io.collator import SpeechCollator

from paddlespeech.s2t.models.ds2 import DeepSpeech2Model

from  matplotlib import pyplot as plt
%matplotlib inline



### 2.2.5 设置预训练模型的路径


In [10]:
config_path = "conf/deepspeech2.yaml" 
checkpoint_path = "./exp/deepspeech/checkpoints/avg_1.pdparams"
audio_file = "data/demo_01_03.wav"


# 读取 conf 文件并结构化
ds2_config = CfgNode(new_allowed=True)
ds2_config.merge_from_file(config_path)
print(ds2_config)



## 2.3 获取特征


### 2.3.1 语音特征介绍
  
#### 2.3.1.1 语音特征提取整体流程图

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/54aefbc16dbf4487a7abe38b0210e5dbf1bb0c74fbe4459f94880a06950269f9" height=1200, width=800  />
<br>
由"莊永松、柯上優 DLHLP - HW1 End-to-end Speech Recognition PPT" 修改得
</div>

#### 2.3.1.2 fbank 提取过程简化图


fbank 特征提取大致可以分为3个步骤：

1. 语音时域信号经过增强，然后进行分帧。

2. 每一帧数据加窗后经过离散傅立叶变换（DFT）得到频谱图。

3. 将频谱图的特征经过 Mel 滤波器得到 logmel fbank 特征。

<div align=center>
<img src="https://ai-studio-static-online.cdn.bcebos.com/08f7ccecc848495599c350aa2c440071b818ba0465734dd29701a2ff149f0a8c" height=1200, width=800 />
<br>
由"DLHLP 李宏毅 语音识别课程PPT" 修改得
</div>

#### 2.3.1.3 CMVN 计算过程

对于所有获取的特征，模型在使用前会使用 CMVN 的方式进行归一化

<div align=center>
  <img src="https://ai-studio-static-online.cdn.bcebos.com/46df63199d88481d9a2713a45ce63d00220e8ac42f9940e886282017758b54bf" height=1200, width=800  />
</div>

 


  
### 2.3.2 构建音频特征提取对象


In [11]:
feat_config = ds2_config.collator
audio_featurizer = AudioFeaturizer(
    spectrum_type=feat_config.spectrum_type,
    feat_dim=feat_config.feat_dim,
    delta_delta=feat_config.delta_delta,
    stride_ms=feat_config.stride_ms,
    window_ms=feat_config.window_ms,
    n_fft=feat_config.n_fft,
    max_freq=feat_config.max_freq,
    target_sample_rate=feat_config.target_sample_rate,
    use_dB_normalization=feat_config.use_dB_normalization,
    target_dB=feat_config.target_dB,
    dither=feat_config.dither)
feature_normalizer = FeatureNormalizer(feat_config.mean_std_filepath) if feat_config.mean_std_filepath else None


  
### 2.3.3 提取音频的特征



In [12]:
# 'None' 只是一个占位符，因为预测的时候不需要reference
speech_segment = SpeechSegment.from_file(audio_file, "None")
audio_feature = audio_featurizer.featurize(speech_segment)
audio_feature_i = feature_normalizer.apply(audio_feature)

audio_len = audio_feature_i.shape[0]
audio_len = paddle.to_tensor(audio_len)
audio_feature = paddle.to_tensor(audio_feature_i, dtype='float32')
audio_feature = paddle.unsqueeze(audio_feature, axis=0)
print(f"shape: {audio_feature.shape}")

plt.figure()
plt.imshow(audio_feature_i.T, origin='lower')
plt.show()


  
## 2.4 使用模型获得结果
    



  
### 2.4.1 构建Deepspeech2模型
  


In [13]:
model_conf = ds2_config.model
# input dim is feature size
model_conf.input_dim = 161
# output_dim is vocab size
model_conf.output_dim = 4301
model = DeepSpeech2Model.from_config(model_conf)


  
### 2.4.2 加载预训练的模型
  


In [14]:
model_dict = paddle.load(checkpoint_path)
model.set_state_dict(model_dict)


  
### 2.4.3 进行预测


In [15]:
decoding_config = ds2_config.decoding
print (decoding_config)
text_feature = TextFeaturizer(unit_type='char',
                            vocab=ds2_config.collator.vocab_filepath)


result_transcripts = model.decode(
        audio_feature,
        audio_len,
        text_feature.vocab_list,
        decoding_method=decoding_config.decoding_method,
        lang_model_path=decoding_config.lang_model_path,
        beam_alpha=decoding_config.alpha,
        beam_beta=decoding_config.beta,
        beam_size=decoding_config.beam_size,
        cutoff_prob=decoding_config.cutoff_prob,
        cutoff_top_n=decoding_config.cutoff_top_n,
        num_processes=decoding_config.num_proc_bsearch)



In [16]:
print ("预测结果为:")
print (result_transcripts[0])


# 3. 总结

* CTC 帮助模型学习语音和 label 之间的 alignment。
* CTC 可以做到帧同步解码，非常适合做流式模型。
* CTC 的输出是之间是独立的，相对于 Seq2Seq 其建模能力差，一般需要外挂 LM 才能得到好的结果。


# 4. 作业 
1. 使用开发模式安装 [PaddleSpeech](https://github.com/PaddlePaddle/PaddleSpeech)  
环境要求：docker, Ubuntu 16.04，root user。  
参考安装方法：[使用Docker安装paddlespeech](https://github.com/PaddlePaddle/PaddleSpeech/blob/develop/docs/source/install.md#hard-get-the-full-funciton-on-your-mechine)

2. 跑通 example/aishell/asr1 中的 conformer 模型，完成训练和预测。 

3. 按照 example 的格式使用自己的数据集训练 ASR 模型。 


# 5. 关注 PaddleSpeech

请关注我们的 [Github Repo](https://github.com/PaddlePaddle/PaddleSpeech/)，非常欢迎加入以下微信群参与讨论：
- 扫描二维码
- 添加运营小姐姐微信
- 通过后回复【语音】
- 系统自动邀请加入技术群


<center><img src="https://ai-studio-static-online.cdn.bcebos.com/87bc7da42bcc401bae41d697f13d8b362bfdfd7198f14096b6d46b4004f09613" width="300" height="300" ></center>


# 5. 参考文献

[1] Graves A, Fernández S, Gomez F, et al. Connectionist temporal classification: labelling unsegmented sequence data with recurrent neural networks[C]//Proceedings of the 23rd international conference on Machine learning. 2006: 369-376.

[2] Graves A, Mohamed A, Hinton G. Speech recognition with deep recurrent neural networks[C]//2013 IEEE international conference on acoustics, speech and signal processing. Ieee, 2013: 6645-6649.

[3] Hannun A, Case C, Casper J, et al. Deep speech: Scaling up end-to-end speech recognition[J]. arXiv preprint arXiv:1412.5567, 2014.

[4] Amodei D, Ananthanarayanan S, Anubhai R, et al. Deep speech 2: End-to-end speech recognition in english and mandarin[C]//International conference on machine learning. PMLR, 2016: 173-182.

[5] Chan W, Jaitly N, Le Q, et al. Listen, attend and spell: A neural network for large vocabulary conversational speech recognition[C]//2016 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP). IEEE, 2016: 4960-4964.

[6] Vaswani A, Shazeer N, Parmar N, et al. Attention is all you need[C]//Advances in neural information processing systems. 2017: 5998-6008.

[7] Zhang Q, Lu H, Sak H, et al. Transformer transducer: A streamable speech recognition model with transformer encoders and rnn-t loss[C]//ICASSP 2020-2020 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP). IEEE, 2020: 7829-7833.

[8] Gulati A, Qin J, Chiu C C, et al. Conformer: Convolution-augmented transformer for speech recognition[J]. arXiv preprint arXiv:2005.08100, 2020.

[9] Retrieved 2021-12-6，from "Sequence Modeling With CTC": https://distill.pub/2017/ctc/#inference

[10] Hannun A Y, Maas A L, Jurafsky D, et al. First-pass large vocabulary continuous speech recognition using bi-directional recurrent dnns[J]. arXiv preprint arXiv:1408.2873, 2014.